# 19. Введение в рекуррентные нейронные сети

# 20. Как рекуррентная нейронная сеть прогнозирует символы

In [25]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import re

from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer


In [26]:
with open('train_data_true.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '')  # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text)  # заменяем все символы кроме кириллицы на пустые символы

In [27]:
#парсим текст, как последовательность символов
num_characters = 34         #33 буквы + пробел
tokenizer =Tokenizer(num_words=num_characters, char_level=True) #токенизируем на уровне символов
tokenizer.fit_on_texts([text]) #формируем токены на основе частности в нашем тексте
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'е': 3, 'т': 4, 'и': 5, 'а': 6, 'н': 7, 'с': 8, 'в': 9, 'р': 10, 'м': 11, 'л': 12, 'ь': 13, 'д': 14, 'п': 15, 'у': 16, 'ы': 17, 'з': 18, 'я': 19, 'б': 20, 'ч': 21, 'к': 22, 'й': 23, 'ж': 24, 'г': 25, 'ш': 26, 'х': 27, 'ю': 28, 'ц': 29, 'щ': 30, 'э': 31, 'ф': 32, 'ъ': 33}


In [28]:
inp_cahrs = 6
data = tokenizer.texts_to_matrix(text) #преобразцем исходный текст в массив OHE
n = data.shape[0] - inp_cahrs #так как мы предсказываем по трем символам - четвертый

In [29]:
X = np.array([data[i:i + inp_cahrs, :] for i in range(n)])
Y = data[inp_cahrs:] #предсказание следующего символа

print(data.shape)

(6307, 34)


In [30]:
model = Sequential()
model.add(Input((inp_cahrs, num_characters))) #при тренировке в рекуррентные модели керас подается сразу вся последовательность
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_2 (SimpleRNN)        │ (None, 128)            │        20,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 34)             │         4,386 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,250 (98.63 KB)

 Trainable params: 25,250 (98.63 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 784us/step - accuracy: 0.1381 - loss: 3.1600
Epoch 2/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step - accuracy: 0.2483 - loss: 2.6751
Epoch 3/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step - accuracy: 0.3093 - loss: 2.4480
Epoch 4/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step - accuracy: 0.3575 - loss: 2.2921
Epoch 5/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step - accuracy: 0.3671 - loss: 2.2118
Epoch 6/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step - accuracy: 0.3874 - loss: 2.1218
Epoch 7/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - accuracy: 0.3931 - loss: 2.0693
Epoch 8/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 906us/step - accuracy: 0.4143 - loss: 1.9949
Epoch 9/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - accuracy: 0.4165 - loss: 1.9898
Epoch 10/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - accuracy: 0.4374 - loss: 1.9284
Epoch 11/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 760us/step - accuracy: 0.4337 - loss: 1.9094
Epoch 12/100
197/19

In [32]:
def buildPharse(inp_str, str_len=50):
    for i in range(str_len):
        x = []
        for j in range(i, i + inp_cahrs):
            x.append(tokenizer.texts_to_matrix(inp_str[j])) #преобразуем символы в One-hot-encoding
        x = np.array(x)
        inp = x.reshape(1, inp_cahrs, num_characters)
        
        pred = model.predict(inp) # предсказываем ОНЕ четвертого символы
        d = tokenizer.index_word[pred.argmax(axis=1)[0]]
        
        inp_str += d #дописываем строку
        
    return inp_str

In [36]:
res = buildPharse('утрене')
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━